In [ ]:
import datetime
import pandas as pd
import numpy as np 
pd.set_option('display.max_columns', None)
kf_logostic = pd.read_excel("itog.xlsx")

def f(x):
    if (x['Опоздание доставка общ'] >= 0):
        return 0
    else:
        return 1
    
def f1(x):
    if (x['Опоздание кухня'] >= 0):
        return 0
    else:
        return 1
    
def f2(x):
    if (x['Опоздание кухня'] < 0):
        return x['Опоздание доставка общ'] - x['Опоздание кухня']
    else:
        return x['Опоздание доставка общ']
    
    
def f3(x):
    if (x['Опоздание только курьеры'] >= 0):
        return 0
    else:
        return 1 

def f4(x):
    if pd.isna(x['Источник заказа']):
        return 'kvikfud.ru'
    else:
        return x['Источник заказа'] 

kf_logostic['Время доставки'] = kf_logostic['Фактическое время доставки'] - kf_logostic['Время открытия']
kf_logostic['Время доставки'] = kf_logostic['Время доставки'].dt.total_seconds()/60

kf_logostic['Опоздание доставка общ'] = kf_logostic['Планируемое время доставки'] - kf_logostic['Фактическое время доставки']
kf_logostic['Опоздание доставка общ'] = kf_logostic['Опоздание доставка общ'].dt.total_seconds()/60

kf_logostic['озвучено время курьерами'] = kf_logostic['Планируемое время доставки'] - kf_logostic['Время открытия']
kf_logostic['озвучено время курьерами'] = kf_logostic['озвучено время курьерами'].dt.total_seconds()/60 - kf_logostic['время заявленное кухней']

kf_logostic['озвучено время курьерами'] = kf_logostic['озвучено время курьерами'].round(decimals=-1)

kf_logostic['Флаг опоздания общ'] = kf_logostic.apply(f, axis=1)
kf_logostic['Флаг опоздания общ'].value_counts()

kf_logostic['Время приготовления'] = kf_logostic['Время окончания приготовления'] - kf_logostic['Время открытия']
kf_logostic['Время приготовления'] = kf_logostic['Время приготовления'].dt.total_seconds()/60

kf_logostic['Опоздание кухня'] = kf_logostic['время заявленное кухней']- kf_logostic['Время приготовления']

kf_logostic['Флаг опоздания кухня'] = kf_logostic.apply(f1, axis=1)
kf_logostic['Флаг опоздания кухня'].value_counts()

kf_logostic = kf_logostic.drop(kf_logostic[kf_logostic['заказы не для учета'] == 1].index)
kf_logostic = kf_logostic.drop(kf_logostic[kf_logostic['Тип заказа'] == 'На время самовывоз'].index)
kf_logostic = kf_logostic.drop(kf_logostic[kf_logostic['Тип заказа'] == 'На время курьером'].index)

kf_deep= kf_logostic.copy()

print('Доставлено заказов вовремя: ', kf_logostic['Флаг опоздания общ'].value_counts()[0])
print('Доставлено заказов с опозданием: ', kf_logostic['Флаг опоздания общ'].value_counts()[1])
print('Процент опозданий заказов: ', round(100*kf_logostic['Флаг опоздания общ'].value_counts()[1]/(kf_logostic['Флаг опоздания общ'].value_counts()[1]+kf_logostic['Флаг опоздания общ'].value_counts()[0]),2),'%')
print('\n')

print('Приготовлено вовремя заказов: ', kf_logostic['Флаг опоздания кухня'].value_counts()[0])
print('Опозданий в приготовлении: ', kf_logostic['Флаг опоздания кухня'].value_counts()[1])
print('Процент опозданий в приготовлении: ', round(100*kf_logostic['Флаг опоздания кухня'].value_counts()[1]/(kf_logostic['Флаг опоздания кухня'].value_counts()[1]+kf_logostic['Флаг опоздания кухня'].value_counts()[0]),2),'%')
print('\n')

kf_logostic_without_bad_closed = kf_logostic.drop(kf_logostic[kf_logostic['заказы не для учета'] == 1].index)
kf_logostic_without_bad_closed = kf_logostic_without_bad_closed.drop(kf_logostic_without_bad_closed[kf_logostic_without_bad_closed['Тип заказа'] == 'Доставка самовывоз'].index)
kf_logostic_without_bad_closed['Опоздание только курьеры'] = kf_logostic_without_bad_closed.apply(f2, axis=1)
kf_logostic_without_bad_closed['Флаг опоздание только курьеры'] = kf_logostic_without_bad_closed.apply(f3, axis=1)

print('Доставлено курьерами вовремя: ', kf_logostic_without_bad_closed['Флаг опоздание только курьеры'].value_counts()[0])
print('Опозданий по вине курьеров: ', kf_logostic_without_bad_closed['Флаг опоздание только курьеры'].value_counts()[1])
print('Процент опозданий по вине курьеров: ', round(100*kf_logostic_without_bad_closed['Флаг опоздание только курьеры'].value_counts()[1]/(kf_logostic_without_bad_closed['Флаг опоздание только курьеры'].value_counts()[1]+kf_logostic_without_bad_closed['Флаг опоздание только курьеры'].value_counts()[0]),2),'%')
print('\n')

#kf_logostic_without_bad_closed['время заявленное кухней'].value_counts()
kf_logostic_without_bad_closed['Источник заказа'] = kf_logostic_without_bad_closed.apply(f4, axis=1)
kf_logostic_without_delivery = kf_logostic_without_bad_closed.drop(kf_logostic_without_bad_closed[kf_logostic_without_bad_closed['Источник заказа'] == 'delivery-club.ru'].index)
print(kf_logostic_without_delivery.groupby('озвучено время курьерами')['Источник заказа'].value_counts())

kf_logostic_without_delivery.loc[(kf_logostic_without_delivery['озвучено время курьерами']>70) | (kf_logostic_without_delivery['озвучено время курьерами']<20)].head(10)

otchet=kf_logostic_without_delivery.loc[(kf_logostic_without_delivery['озвучено время курьерами']>70) | (kf_logostic_without_delivery['озвучено время курьерами']<20)]
otchet.to_excel('проблемные заказы.xlsx')

In [ ]:
kf_deep['Day of Week'] = kf_deep['Время открытия'].dt.dayofweek
kf_deep['Day of Week'].replace({0: "Monday", 1: "Tuesday", 2: "Wednesday", 3: "Thursday", 4: "Friday", 5: "Saturday", 6: "Sunday"}, inplace=True)
kf_deep['Hour'] = kf_deep['Время открытия'].dt.hour
kf_deep = kf_deep.drop(kf_deep[kf_deep['Тип заказа'] == 'На время самовывоз'].index)
kf_deep = kf_deep.drop(kf_deep[kf_deep['Тип заказа'] == 'Доставка самовывоз'].index)
kf_deep['Стоял на точке'] = kf_deep['Время отправки доставки'] - kf_deep['Время окончания приготовления'] 
kf_deep['Стоял на точке'] = kf_deep['Стоял на точке'].dt.total_seconds()/60
bins = [0, 10, 20, 30, 40, 50, 60, 1000]
labels = ['<10', '10-20', '20-30', '30-40', '40-50', '50-60', '>60']
cats = pd.cut(kf_deep['Стоял на точке'], bins, labels=labels)
pd.value_counts(cats)
kf_deep['Bins']=cats

def bins(kf_deep_x, point):
    kf_deep = kf_deep_x.loc[kf_deep_x['Торговое предприятие'] == point] 
    return kf_deep['Bins'].value_counts()  

def bins_day(kf_deep_x, point, day, start, end):
    kf_deep = kf_deep_x.loc[(kf_deep_x['Торговое предприятие'] == point) & (kf_deep_x['Day of Week']==day) & (kf_deep_x['Hour'] >=start) & (kf_deep_x['Hour'] <end)] 
    return kf_deep['Bins'].value_counts()

def table_day(kf_deep_x, point="всем точкам"):
    if point=="всем точкам":
        kf_deep = kf_deep_x
    else:
        kf_deep = kf_deep_x.loc[(kf_deep_x['Торговое предприятие'] == point)]
    print('Данные по', point, 'Сколько заказов ожидали отправки')
    grouped = kf_deep.groupby('Day of Week').Bins.value_counts().unstack(0)[["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]].fillna(0.0).astype(int)
    print(grouped, '\n')
    grouped[3:].plot(kind='bar', title = "Данные по " + point, xlabel='Время ожидания на точке', ylabel='Количество заказов')
    return kf_deep['Bins'].value_counts()

final_table_amount_orders = pd.DataFrame(index=['<10', '10-20', '20-30', '30-40', '40-50', '50-60', '>60'])
final_table_amount_orders=final_table_amount_orders.join(bins(kf_deep, 'Алроса'))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Алр всего'})
final_table_amount_orders=final_table_amount_orders.join(bins_day(kf_deep, 'Алроса', "Friday", 17,21))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Алр Fri 17-20'})
final_table_amount_orders=final_table_amount_orders.join(bins_day(kf_deep, 'Алроса', "Saturday", 17,21))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Алр Sat 17-20'})
final_table_amount_orders=final_table_amount_orders.join(bins(kf_deep, 'Горького'))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Гор всего'})
final_table_amount_orders=final_table_amount_orders.join(bins_day(kf_deep, 'Горького', "Friday", 17,21))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Гор Fri 17-20'})
final_table_amount_orders=final_table_amount_orders.join(bins_day(kf_deep, 'Горького', "Saturday", 17,21))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Гор Sat 17-20'})
final_table_amount_orders=final_table_amount_orders.join(bins(kf_deep, 'Раздольная'))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Разд всего'})
final_table_amount_orders=final_table_amount_orders.join(bins_day(kf_deep, 'Раздольная', "Friday", 17,21))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Разд Fri 17-20'})
final_table_amount_orders=final_table_amount_orders.join(bins_day(kf_deep, 'Раздольная', "Saturday", 17,21))
final_table_amount_orders=final_table_amount_orders.rename(columns={'Bins': 'Разд Sat 17-20'})

table_day(kf_deep)
table_day(kf_deep, 'Алроса')
table_day(kf_deep, 'Горького')
table_day(kf_deep, 'Раздольная')

print(final_table_amount_orders)

#Cooking speed
kf_deep.head()

def cooking_time(kf_deep_x, point, day, start, end):
    kf_deep = kf_deep_x.loc[(kf_deep_x['Торговое предприятие'] == point) & (kf_deep_x['Day of Week']==day) & (kf_deep_x['Hour'] >=start) & (kf_deep_x['Hour'] <end)] 
    return kf_deep['Время приготовления'].mean(), kf_deep['Время приготовления'].max(), kf_deep['Флаг опоздания кухня'].sum(), kf_deep['Флаг опоздания кухня'].count(), kf_deep['стоимость заказа'].sum() 

cooking_array = np.array([['Алроса', 'Monday', 12, 14],
                 ['Алроса', 'Tuesday', 12, 14],
                 ['Алроса', 'Wednesday', 12, 14],
                 ['Алроса', 'Thursday', 12, 14],
                 ['Алроса', 'Friday', 12, 14],
                 ['Алроса', 'Saturday', 12, 14],
                 ['Алроса', 'Sunday', 12, 14],
                 ['Алроса', 'Monday', 17, 21],
                 ['Алроса', 'Tuesday', 17, 21],
                 ['Алроса', 'Wednesday', 17, 21],
                 ['Алроса', 'Thursday', 17, 21],
                 ['Алроса', 'Friday', 17, 21],
                 ['Алроса', 'Saturday', 17, 21],
                 ['Алроса', 'Sunday', 17, 21],
                 ['Горького', 'Tuesday', 12, 14],
                 ['Горького', 'Wednesday', 12, 14],
                 ['Горького', 'Thursday', 12, 14],
                 ['Горького', 'Friday', 12, 14],
                 ['Горького', 'Saturday', 12, 14],
                 ['Горького', 'Sunday', 12, 14],
                 ['Горького', 'Monday', 17, 21],
                 ['Горького', 'Tuesday', 17, 21],
                 ['Горького', 'Wednesday', 17, 21],
                 ['Горького', 'Thursday', 17, 21],
                 ['Горького', 'Friday', 17, 21],
                 ['Горького', 'Saturday', 17, 21],
                 ['Горького', 'Sunday', 17, 21],
                 ['Раздольная', 'Tuesday', 12, 14],
                 ['Раздольная', 'Wednesday', 12, 14],
                 ['Раздольная', 'Thursday', 12, 14],
                 ['Раздольная', 'Friday', 12, 14],
                 ['Раздольная', 'Saturday', 12, 14],
                 ['Раздольная', 'Sunday', 12, 14],
                 ['Раздольная', 'Monday', 17, 21],
                 ['Раздольная', 'Tuesday', 17, 21],
                 ['Раздольная', 'Wednesday', 17, 21],
                 ['Раздольная', 'Thursday', 17, 21],
                 ['Раздольная', 'Friday', 17, 21],
                 ['Раздольная', 'Saturday', 17, 21],
                 ['Раздольная', 'Sunday', 17, 21]])

label = []
max_array = []
min_array = []
mean_array = []
lateness_array = []
receipts_array = []
ours_array = []
sum_array = []
for i in cooking_array:
    mean, max, lateness, receipts, sum = cooking_time(kf_deep, str(i[0]), str(i[1]), int(i[2]), int(i[3]))
    label.append(str(i[0]) + ' ' + str(i[2]) + '-' + str(i[3]) + ' ' + str(i[1])[:3])
    ours_array.append(int(i[3])-int(i[2]))
    mean_array.append(int(mean))
    max_array.append(int(max))
    lateness_array.append(lateness)
    receipts_array.append(receipts)
    sum_array.append(sum)
    
df = pd.DataFrame(data=mean_array, index=label) 
df.columns =['Среднее время']
df['Максимальное'] = max_array
df['Опоздания кухни'] = lateness_array
df['Чеков всего'] = receipts_array
df['Сумма чеков'] = sum_array
df['Чеков в час'] = np.array(receipts_array)/np.array(ours_array)
df['Чеков в час'] = df['Чеков в час'].round(decimals = 0).astype(int)
df['Сумма в час'] = np.array(sum_array)/np.array(ours_array)
df['Сумма в час'] = df['Сумма в час'].round(decimals = 0).astype(int)
df.sort_values(by=['Среднее время'], ascending=False).style.background_gradient(cmap='Blues')
